<a href="https://colab.research.google.com/github/KARTHIKEYANSH55/BeagleBoneBlackRootfsUbuntu-16.04/blob/master/1_farm_building_blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FARM Building Blocks

Welcome to the FARM building blocks tutorial! There are many different ways to make use of this repository, but in this notebook, we will be going through the most import building blocks that will help you harvest the rewards of a successfully trained NLP model.

Happy FARMing!

## 1) Text Classification

GermEval 2018 (GermEval2018) (https://projects.fzai.h-da.de/iggsa/) is an open data set containing texts that need to be classified by whether they are offensive or not. There are a set of coarse and fine labels, but here we will only be looking at the coarse set which labels each example as either OFFENSE or OTHER. To tackle this task, we are going to build a classifier that is composed of Google's BERT language model and a feed forward neural network prediction head.

### Setup

In [0]:
# Install FARM
!pip install farm==0.3.2

     |████████████████████████████████| 102kB 2.9MB/s 
     |████████████████████████████████| 47.7MB 60kB/s 
     |████████████████████████████████| 317kB 56.1MB/s 
     |████████████████████████████████| 2.5MB 42.1MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 92kB 14.2MB/s 
     |████████████████████████████████| 1.1MB 38.1MB/s 
     |████████████████████████████████| 153kB 60.0MB/s 
     |████████████████████████████████| 460kB 55.9MB/s 
     |████████████████████████████████| 870kB 50.9MB/s 
     |████████████████████████████████| 1.0MB 48.5MB/s 
     |████████████████████████████████| 51kB 339kB/s 
     |████████████████████████████████| 81kB 13.6MB/s 
     |████████████████████████████████| 204kB 55.3MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
  Created wheel for farm: filename=farm-0.3.2-cp36-none-any.whl size=96130 sha256=6a27e4f9006f97dc8d3c9f989ace051d5bab8aa311e06cbcadfb07f3214711f3
  Stored in directory:

In [0]:
# Here are the imports we need

import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger


03/13/2020 14:57:39 - INFO - transformers.file_utils -   PyTorch version 1.4.0 available.
03/13/2020 14:57:40 - INFO - transformers.modeling_xlnet -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
03/13/2020 14:57:40 - WARNING - farm.train -   Apex not installed. If you use distributed training with local rank != -1 apex must be installed.


In [0]:
# Farm allows simple logging of many parameters & metrics. Let's use the MLflow framework to track our experiment ...
# You will see your results on https://public-mlflow.deepset.ai/
ml_logger = MLFlowLogger(tracking_uri="https://public-mlflow.deepset.ai/")
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab")



 __          __  _                            _        
 \ \        / / | |                          | |       
  \ \  /\  / /__| | ___ ___  _ __ ___   ___  | |_ ___  
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \ 
    \  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) |
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/ 
  ______      _____  __  __  
 |  ____/\   |  __ \|  \/  |              _.-^-._    .--.
 | |__ /  \  | |__) | \  / |           .-'   _   '-. |__|
 |  __/ /\ \ |  _  /| |\/| |          /     |_|     \|  |
 | | / ____ \| | \ \| |  | |         /               \  |
 |_|/_/    \_\_|  \_\_|  |_|        /|     _____     |\ |
                                     |    |==|==|    |  |
|---||---|---|---|---|---|---|---|---|    |--|--|    |  |
|---||---|---|---|---|---|---|---|---|    |==|==|    |  |
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 


In [0]:
# We need to fetch the right device to drive the growth of our model
# Make sure that you have gpu turned on in this notebook by going to
# Runtime>Change runtime type and select GPU as Hardware accelerator.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Devices available: cuda


### Data Handling

In [0]:
# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

tokenizer = Tokenizer.load(
    pretrained_model_name_or_path="bert-base-cased",
    do_lower_case=False)

03/13/2020 14:57:43 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizer'
03/13/2020 14:57:44 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmp2kp092x1
100%|██████████| 213450/213450 [00:00<00:00, 393957.96B/s]
03/13/2020 14:57:45 - INFO - transformers.file_utils -   copying /tmp/tmp2kp092x1 to cache at /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
03/13/2020 14:57:45 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
03/13/2020 14:57:45 - INFO - transformers.file_utils -   removing temp file /tmp/tmp2kp092x1
03/13/2020 14:57:45 - INFO 

In [0]:
tokenizer.tokenize("I am about  to ride the train without  a ticket YOLO!")

['I',
 'am',
 'about',
 'to',
 'ride',
 'the',
 'train',
 'without',
 'a',
 'ticket',
 'Y',
 '##OL',
 '##O',
 '!']

In [0]:
# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor

processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir="../data/germeval18",
                                        label_list = ["OTHER", "OFFENSE"],
                                        metric = "f1_macro",
                                        label_column_name = "coarse_label")

In [0]:
# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

12/10/2019 15:12:19 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
12/10/2019 15:12:19 - INFO - farm.data_handler.data_silo -   Loading train set from: ../data/germeval18/train.tsv 
12/10/2019 15:12:19 - INFO - farm.data_handler.utils -    Couldn't find ../data/germeval18/train.tsv locally. Trying to download ...
12/10/2019 15:12:19 - INFO - farm.data_handler.utils -   downloading and extracting file germeval18 to dir /data
100%|██████████| 525101/525101 [00:01<00:00, 399207.47B/s]
12/10/2019 15:12:22 - INFO - farm.data_handler.data_silo -   Got ya 2 parallel workers to convert 5009 dictionaries to pytorch datasets (chunksize = 501)...
12/10/2019 15:12:22 - INFO - farm.data_handler.data_silo -    0    0 
12/10/2019 15:12:22 - INFO - farm.data_handler.data_silo -   /w\  /w\
12/10/2019 15:12:22 - INFO - farm.data_ha

### Modeling

In FARM, we make a strong distinction between the language model and prediction head so that you can mix and match different building blocks for your needs.

For example, in the transfer learning paradigm, you might have the one language model that you will be using for both document classification and NER. Or you perhaps you have a pretrained language model which you would like to adapt to your domain, then use for a downstream task such as question answering. 

All this is possible within FARM and requires only the replacement of a few modular components, as we shall see below.

Let's first have a look at how we might set up a model.

In [0]:
# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "bert-base-german-cased"

language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

12/10/2019 15:22:19 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-german-cased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpmrg4qz0m
100%|██████████| 288/288 [00:00<00:00, 161859.78B/s]
12/10/2019 15:22:20 - INFO - transformers.file_utils -   copying /tmp/tmpmrg4qz0m to cache at /root/.cache/torch/transformers/e653e2fe0970d519c5a3b6c0286e1630ad2f0eade78f82b4916ec945d6f06d48.958be95f8721c8cc3ff0998c94a7b77083ecd9345f62423db24ad2387d599c7d
12/10/2019 15:22:20 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/e653e2fe0970d519c5a3b6c0286e1630ad2f0eade78f82b4916ec945d6f06d48.958be95f8721c8cc3ff0998c94a7b77083ecd9345f62423db24ad2387d599c7d
12/10/2019 15:22:20 - INFO - transformers.file_utils -   removing temp file /tmp/tmpmrg4qz0m
12/10/2019 15:22:20 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.h

In [0]:
# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

LAYER_DIMS = [768, 2]

prediction_head = TextClassificationHead(layer_dims=LAYER_DIMS)

In [0]:
# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

### Training

In [0]:
# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

LEARNING_RATE = 2e-5
WARMUP_PROPORTION = 0.1
N_EPOCHS = 1

optimizer, warmup_linear = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    warmup_proportion=WARMUP_PROPORTION,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

12/10/2019 15:25:34 - INFO - farm.modeling.optimization -   Number of optimization steps: 141


In [0]:
# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    warmup_linear=warmup_linear,
    device=device,
)

In [0]:
model = trainer.train(model)

12/10/2019 15:25:36 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 16/16 [00:07<00:00,  2.06it/s]12/10/2019 15:28:25 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVA

In [0]:
# Test your model on a sample (Inference)
from farm.infer import Inferencer
from pprint import PrettyPrinter

infer_model = Inferencer(processor=processor, model=model, gpu=True)

basic_texts = [
    {"text": "Martin ist ein Idiot"},
    {"text": "Martin Müller spielt Handball in Berlin"},
]
result = infer_model.inference_from_dicts(dicts=basic_texts)
PrettyPrinter().pprint(result)


12/10/2019 15:31:12 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
12/10/2019 15:31:12 - INFO - farm.infer -   Got ya 1 parallel workers to do inference on 2dicts (chunksize = 4)...
12/10/2019 15:31:12 - INFO - farm.infer -    0 
12/10/2019 15:31:12 - INFO - farm.infer -   /w\
12/10/2019 15:31:12 - INFO - farm.infer -   /'\
12/10/2019 15:31:12 - INFO - farm.infer -   
  0%|          | 0/2 [00:00<?, ? Dicts/s]12/10/2019 15:31:12 - INFO - farm.data_handler.processor -   *** Show 3 random examples ***
12/10/2019 15:31:12 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
__

[{'predictions': [{'context': 'Martin ist ein Idiot',
                   'end': None,
                   'label': 'OFFENSE',
                   'probability': 0.7827183,
                   'start': None},
                  {'context': 'Martin Müller spielt Handball in Berlin',
                   'end': None,
                   'label': 'OTHER',
                   'probability': 0.87313026,
                   'start': None}],
  'task': 'text_classification'}]


# Switch to NER

In a transfer learning paradigm, there is a core computation that is shared amongst all tasks. FARM's modular structure means that you can easily swap out different building blocks to make the same language model work for many different tasks.

We can adapt the above text classification model to NER by simply switching out the processor and prediction head.

In [0]:
# Import the new building blocks

from farm.data_handler.processor import NERProcessor
from farm.modeling.prediction_head import TokenClassificationHead
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab_NER")


In [0]:
# This processor will preprocess the data for the CoNLL03 NER task
ner_labels = ["[PAD]", "X", "O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-OTH", "I-OTH"]

ner_processor = NERProcessor(tokenizer=tokenizer, 
                             max_seq_len=128, 
                             data_dir="../data/conll03-de",
                             label_list=ner_labels,
                             metric="seq_f1")

In [0]:
# This prediction head is also a feed forward neural network but expects one
# vector per token in the input sequence and will generate a set of logits
# for each input

LAYER_DIMS = [768, 13]

ner_prediction_head = TokenClassificationHead(layer_dims=LAYER_DIMS)

In [0]:
# We can integrate these new pieces with the rest using this code
# It is pretty much the same structure as what we had above for text classification

BATCH_SIZE = 32
EMBEDS_DROPOUT_PROB = 0.1
LEARNING_RATE = 2e-5
WARMUP_PROPORTION = 0.1
N_EPOCHS = 1
N_GPU = 1

data_silo = DataSilo(
    processor=ner_processor,
    batch_size=BATCH_SIZE)

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[ner_prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_token"],
    device=device)

optimizer, warmup_linear = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    warmup_proportion=WARMUP_PROPORTION,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

trainer = Trainer(
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    warmup_linear=warmup_linear,
    device=device,
)

12/09/2019 15:38:43 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
12/09/2019 15:38:43 - INFO - farm.data_handler.data_silo -   Loading train set from: ../data/conll03-de/train.txt 
12/09/2019 15:38:43 - INFO - farm.data_handler.utils -    Couldn't find ../data/conll03-de/train.txt locally. Trying to download ...
12/09/2019 15:38:43 - INFO - farm.data_handler.utils -   downloading and extracting file conll03-de to dir /data
12/09/2019 15:38:46 - INFO - farm.data_handler.data_silo -   Got ya 2 parallel workers to convert 24000 dictionaries to pytorch datasets (chunksize = 2000)...
12/09/2019 15:38:46 - INFO - farm.data_handler.data_silo -    0    0 
12/09/2019 15:38:46 - INFO - farm.data_handler.data_silo -   /w\  /w\
12/09/2019 15:38:46 - INFO - farm.data_handler.data_silo -   / \  /'\
12/09/2019 15:38:46 - INFO -

In [0]:
model = trainer.train(model)

12/09/2019 15:39:50 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 69/69 [00:23<00:00,  2.92it/s]12/09/2019 15:41:56 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVA